In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import csv
from string import punctuation

In [101]:
topics = ['AltS','ARWar','Bees','Bombing','Evolution','NASA','Orcas','Sangre','SunTzu','Tornado'] #For file views

regressionstat = pd.DataFrame(index = topics , columns = ['Ndoc','R2','MSE','MAE'])
querydoc = pd.DataFrame(columns = ['queryterm','urlid'])
docindex = 0
for topic in topics:    
    print(topic)
    dest = "Ressource\\" + topic
    
    #Note:When we wanted to consider the document dwell time, we checked the page-viewed and noticed that the file is missing some documents read by the user (ex. 36437557)
    #Note:We comment this, and extract the user behaviour from page-viewed file
    cols_queryclick = ['userID','queryID','queryterm','query_time','click_time','rank','url','title','desc']
    behaviour_nasa = pd.read_csv('Raw Data\\'+ topic + '_query-and-click.csv',names=cols_queryclick)
    behaviour_nasa['queryterm'] = [content.lower() for content in behaviour_nasa['queryterm'] ] #Lowercase query
    behaviour_nasa['queryterm'] = [content.strip() for content in behaviour_nasa['queryterm'] ] #Remove space at the beginning and end of query
    behaviour_nasa['queryterm'] = [content.strip(punctuation) for content in behaviour_nasa['queryterm']] #Remove leading or training punctuation
 
#     display(behaviour_nasa)
#     print("Before merge")

    score_nasa = pd.read_csv('Raw Data\\'+ topic + '_test-score.csv',names=['userID','Before','After'])
    score_nasa['Gain']=score_nasa['After']-score_nasa['Before']
    score_nasa[['Before','After','Gain']] *= 100 #Multiply by 100 because it is easier, especially with the individual information gain
    
#Index URLS: Give an ID for every url
#     behaviour_nasa = queryclick_nasa[['userID','queryterm','url']].dropna()
    unique = behaviour_nasa['url'].unique()
    unique = unique[1:]

    urlindexx= pd.DataFrame(unique) #pop zero to remove NaN first value
    urlindexx['urlid'] = 'd' + (1+urlindexx.index).astype('string')
    urlindexx.columns = ['url','urlid']
    #Add docid to behvaviour nasa
    behaviour_nasa = behaviour_nasa.merge(urlindexx, on='url')

         ##FILTER USERS
    #Remove those who did not see any doc (disregarding if they have a gain or not)
    common = behaviour_nasa.merge(score_nasa, on=["userID"])
    users_nodoc = score_nasa[~score_nasa.userID.isin(common.userID)]
    negatives =  score_nasa[score_nasa.Gain < 0 ]
    print("negatives are")
    print(negatives)
    
    behaviour_nasa = behaviour_nasa[~behaviour_nasa.userID.isin(users_nodoc['userID'])].copy() 
    behaviour_nasa = behaviour_nasa[~behaviour_nasa.userID.isin(negatives['userID'])].copy() 
    display(behaviour_nasa)
    score_nasa = score_nasa[score_nasa['Gain'] >= 0]
    score_nasa = score_nasa[~score_nasa.userID.isin(users_nodoc)].copy()
    score_nasa.to_csv(dest + "\\test-score.csv", index=False, float_format="%.2f")
 
  
    
##Get the time spend on each page from the pageview file
    cols_pageviewed = ['userID','url','timeclicked','referrer','stay_time','active_time','time_out']
    pageviewed_nasa = pd.read_csv('Raw Data\\'+ topic + '_page-viewed.csv',names=cols_pageviewed)
    pageviewed_nasa['url'] = pageviewed_nasa['url'].replace('.secure.waps.io','',regex= True)
    pageviewed_nasa['url'] = pageviewed_nasa['url'].replace('.waps.io','',regex= True)
    pageviewed_nasa = pageviewed_nasa[pageviewed_nasa['url'].str.contains('uid=')==False] 
    pageviewed_nasa = pageviewed_nasa[pageviewed_nasa['url'].str.contains('query=')==False] 
    behaviour_nasa['url'] = behaviour_nasa['url'].replace('https','http',regex= True).copy()
    behaviour_nasa['url'] = behaviour_nasa['url'].replace('https','http',regex= True).copy()
    #Remove docs that are in page-viewed but not in query-click. Those are documents that were refered pages other than the search engine
    pageviewed_nasadoc = pageviewed_nasa[pageviewed_nasa['url'].isin(behaviour_nasa['url']) & pageviewed_nasa['userID'].isin(behaviour_nasa['userID'])].copy()

    pageviewed_nasadoc = pageviewed_nasa[(pageviewed_nasa['userID'].astype(str)+pageviewed_nasa['url']).isin(behaviour_nasa['userID'].astype(str)+behaviour_nasa['url'])]
    
    behaviour_nasa['active_time'] = -1
    print(len(behaviour_nasa))
    
    
    userlog = []
    count = -1
    print("size of beh_nasa", len(behaviour_nasa))
    behaviour_nasa = behaviour_nasa.reset_index(drop=True)
    for i in behaviour_nasa.index:
        try: #When a row is deleted, the index is removed
            print("index is:", i)
            user = behaviour_nasa['userID'][i]
            url = str(behaviour_nasa['url'][i])
            print(user)
            
            if str(user)+url not in userlog:
                userlog.append(str(user)+url)
                count = 0
                print("First time user")
            else:
                count += 1
                print("Hi again")
            active_time = pageviewed_nasadoc.active_time[(pageviewed_nasadoc['userID']==user) & (pageviewed_nasadoc['url']== url)]        
            if(user ==39033631):
                print("HOLA")
                display(active_time)
            if(i == 4):
                print(user)
                display(active_time)
                print("negative user")
                
            
            if(len(active_time) == 0 ): #Some documents are in behaviour_nasa but not in querydocnasa
                behaviour_nasa['active_time'].iloc[i] = -2 #convert from milliseconds to minutes
#                
            elif(len(active_time) == 1): #User visited the url one time --> one active time is associated
                 behaviour_nasa['active_time'].iloc[i] = active_time / 60000 #convert from milliseconds to minutes
            else:
                print("Count is", count)
                active_time = active_time.iloc[count]
                behaviour_nasa['active_time'].iloc[i] = active_time / 60000 #convert from milliseconds to minutes

            if(user ==39033631):
             print("assigned")
             display(behaviour_nasa['active_time'][i])
        except Exception as e:
#             print(" exception for user,", user)
#             print(str(e))
#             print("except index", i)
#           
            continue
                       
#     behaviour_nasa.drop([behaviour_nasa['active_time']==-2])

    behaviour_nasa = behaviour_nasa.drop(behaviour_nasa[behaviour_nasa['active_time'] < 0 ].index) #Those who have -1 value (from except) and those -2


   
    ##Filter Query Click
    queryclick_nasa = behaviour_nasa[behaviour_nasa.userID.isin(score_nasa.userID)].copy()
    queryclick_nasa.to_csv(dest + "\\query-click.csv", index=False)

    queryclick_nasa = queryclick_nasa.set_index("userID")

##Generate Query Behaviour
    query_beh = queryclick_nasa.copy()
  
    query_beh = query_beh[query_beh['rank']>0].copy() #drop queries that did not result in any viewing doc
    
    query_beh =  query_beh.drop(columns=['queryID','query_time','click_time','rank','url','title','desc'])
    
    query_beh = query_beh.reset_index()
    
    query_beh =  query_beh.groupby('userID').agg({'queryterm': '+'.join})
    query_beh = query_beh.queryterm.str.split('+',expand=True,)
    query_beh.columns = query_beh.columns + 1
    query_beh = query_beh.apply(lambda col: col.str.lower()) #Lowercase query
    query_beh = query_beh.apply(lambda col: col.str.strip()) #Remove space at the beginning and end of query
    query_beh = query_beh.apply(lambda col: col.str.strip(punctuation)) #Remove leading or training punctuation
    query_beh.to_csv(dest + "\\user-querybehaviour.csv", index=True)
   
      
##Models Creation
    #Get Pivot for Regression #Matrix
    urlpivot_nasa = behaviour_nasa[['userID','urlid','active_time']]

    urlpivot_nasa["value"]= 1 #Use this if I want binary model, and change "values = "

    #The pivot has the time spent per document
    urlpivot_nasa = pd.pivot_table(urlpivot_nasa, index = 'userID', values = 'active_time', columns='urlid', aggfunc = 'sum').fillna(0)
    urlpivot_nasa = urlpivot_nasa.merge(score_nasa[['userID','Gain']],on = 'userID', how='left')
    urlpivot_nasa = urlpivot_nasa.drop(urlpivot_nasa[urlpivot_nasa.Gain < 0].index) #Drop negative gains
    urlpivot_nasa = pd.DataFrame(urlpivot_nasa.to_records())


    #Regression
    import statsmodels.api as sm
    from sklearn.metrics import mean_squared_error
    import math
    X=urlpivot_nasa.drop(['Gain','index'], axis = 1).dropna().set_index('userID')
    y=urlpivot_nasa[['Gain','userID']].dropna().set_index('userID')
    regr= linear_model.LinearRegression(positive = True, fit_intercept = True, normalize = False)
    try:
     model_nasa = regr.fit(X,y)
    except:
     model_nasa = regr.fit(X,y)
    # # print("Coef = ", model_nasa.coef_)
    print("Intercept = ", model_nasa.intercept_)

    #### Regression2 
    #Save result in a dataframe and mergeit with urlindex
    coef_nasa = pd.DataFrame(model_nasa.coef_[0],X.columns).reset_index()
    coef_nasa.columns= ['urlid','coef']

    urlindexx = coef_nasa.merge(urlindexx, on='urlid' , how='left').set_index('urlid').copy()
    pd.options.display.float_format = '{:20,.2f}'.format #To remove the scientific notation
    urlindexx['coef'] = urlindexx['coef'].round(9) #To remove the scientific notation
    urlindex = urlindexx.copy()
    urlindex['coef'].loc[urlindex['coef']>100] = 100
    urlindex['coef'].loc[urlindex['coef']<-100] = -100
    # urlindex.coef[urlindex['coef']<0] = 0   #Consider negative individual gain zero
    
    #Model evaluation
    import statsmodels.api as sm
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error as mae
    import math
    print("Documents in the index: " + str(urlindexx.shape[0]))
    #     print("The total number of users is: " + str(score_nasa.shape[0]))
    #     print(" Users with no doc: " + str(users_nodoc.shape[0]))
    # print("The number of Zero users: " + str(zero_users.shape[0]))
    y_pred = model_nasa.predict(X)
    y_pred = pd.DataFrame(y_pred.round(3))
    y_pred.index = y.index
    regressionstat["Ndoc"][topic]= str(urlindex.shape[0])
    regressionstat["R2"][topic]= r2_score(y, y_pred)
    regressionstat["MSE"][topic] = mean_squared_error(y, y_pred)
    regressionstat["MAE"][topic] = mae(y, y_pred)
    print(mean_squared_error(y, y_pred))
    print(math.sqrt(mean_squared_error(y, y_pred)))
    print(mae(y, y_pred))
    urlindex.to_csv(dest + "\\doc-gain.csv", index=True , float_format="%.2f")
    print(model_nasa.coef_[0])
    
##Baseline Generation    
    #Get behaviour Baseline (Threshold th) behaviour th and draw plot

    behaviour_th_doc = behaviour_nasa.filter(['userID', 'urlid']).groupby('userID').sum()
    behaviour_th_doc = behaviour_th_doc.urlid.str.split('d',expand=True,)
    behaviour_th_doc = 'd' + behaviour_th_doc
    display(behaviour_th_doc)
    behaviour_th_doc = behaviour_th_doc.drop(behaviour_th_doc.columns[0], axis=1)

    behaviour_th_doc.to_csv(dest + "\\user-docbehaviour.csv", index=True) #If the user read the same doc twice, the file will show the doc twice
    behaviour_th_gain = pd.DataFrame(index=behaviour_th_doc.index,columns=behaviour_th_doc.columns)

    #Get the gain , document by document for every user. 
    behaviour_nasa["gain"] = 0.0
    behaviour_nasa["docgain"] = 0.0
#     behaviour_nasa["active_time_indiv"] = 0.0
    for i in behaviour_nasa.index:
         behaviour_nasa["docgain"][i] = (urlindex['coef'][behaviour_nasa["urlid"][i]]).copy()
         behaviour_nasa["gain"][i] = (urlindex['coef'][behaviour_nasa["urlid"][i]] * behaviour_nasa["active_time"][i]).copy()

    behaviour_th_gain = behaviour_nasa.copy().drop(columns=['queryterm','url','urlid','active_time','docgain'])

    behaviour_th_gain["gain"] = behaviour_th_gain["gain"].round(2).astype(str)
    behaviour_th_gain  = behaviour_th_gain.groupby('userID').agg(lambda x : ', '.join(x))
    behaviour_th_gain["gain"] = str(model_nasa.intercept_[0].round(2)) + "," + behaviour_th_gain["gain"]
    behaviour_th_gain = behaviour_th_gain.gain.str.split(',',expand=True,)
    behaviour_th_gain.to_csv(dest + "\\user-docgain.csv", index=True)
   
    score_nasa = score_nasa.set_index('userID')

    behaviour_th_cum = behaviour_th_gain.copy()

    #Add the intercept of the model for the first gain #Note: I added it in the doc-gain
#     behaviour_th_cum[1] = behaviour_th_cum[1] + model_nasa.intercept_
    behaviour_th_cum = behaviour_th_cum.round(4).astype(float)
    behaviour_th_cum = behaviour_th_cum.cumsum(axis = 1)

    behaviour_th_cum.to_csv(dest + "\\user-cumgain.csv", index=True, float_format="%.2f")

## Querydoc generation
    #Docquery index
    queryindex = behaviour_nasa[["queryterm","urlid"]] 
    queryindex = queryindex.drop_duplicates(subset = ['urlid', 'queryterm']) 
    queryindex = queryindex.groupby('queryterm').agg({'urlid': ','.join})

    

    queryindex = queryindex.reset_index()


    print("Here 2")
    display(queryindex)
#     queryindex.to_csv(dest + "\\querydoc_index.csv", index=False
# #                       ,sep = ','
#                       ,quoting=csv.QUOTE_NONE
#     , )
    queryindex.to_csv(dest + "\\querydoc_index.csv", index=False,quoting = csv.QUOTE_NONE,escapechar = ',')
display(regressionstat)

AltS
negatives are
      userID               Before                After                 Gain
2   43164637                68.42                52.63               -15.79
3   41914129                68.42                63.16                -5.26
14  43584185                52.63                 0.00               -52.63
16  42861604                36.84                26.32               -10.53
25  43150296                84.21                78.95                -5.26
27  43588858                52.63                47.37                -5.26
31  40613469                26.32                 0.00               -26.32
38  39119481                63.16                57.89                -5.26
45  43405953                47.37                31.58               -15.79


,userID,queryID,queryterm,query_time,click_time,rank,url,title,desc,urlid
0,41683720,349,sintomas causas y prevencion del mal de altura,2017-07-22 10:53:41.0,2017-07-22 10:53:55.0,2.00,https://infotratamiento.com/consejos-salud/mal...,Qué es el <b>Mal</b> <b>de</b> <b>Altura</b>? ...,<b>C ausas del mal de altura</b>. La presión a...,d1
1,11311952,389,"síntomas, causas y prevención del mal de altura",2017-07-22 11:12:31.0,2017-07-22 11:12:41.0,2.00,https://infotratamiento.com/consejos-salud/mal...,Qué es el <b>Mal</b> <b>de</b> <b>Altura</b>? ...,<b>Síntomas del mal de altura</b>. Hay cinco m...,d1
2,41683720,349,sintomas causas y prevencion del mal de altura,2017-07-22 10:53:41.0,2017-07-22 11:04:57.0,7.00,http://cae.ar.tripod.com/Publicaciones/prevenc...,<b>PREVENCION</b> <b>Y</b> TRATAMIENTO <b>DEL<...,<b>PREVENCION Y</b> TRATAMIENTO <b>DEL MAL DE ...,d2
3,41683720,349,sintomas causas y prevencion del mal de altura,2017-07-22 10:53:41.0,2017-07-22 11:05:38.0,9.00,http://www.sos-emergencias.es/document/malaltu...,<b>PREVENCION</b> <b>Y</b> TRATAMIENTO <b>DEL<...,Durante las fase <b>de</b> reposo en los campo...,d3
4,43665081,418,"sintomas, causas y prevencion del mal de altura",2017-07-22 12:13:17.0,2017-07-22 12:13:56.0,3.00,https://es.wikipedia.org/wiki/Mal_de_monta%C3%B1a,"<b>Mal</b> <b>de</b> montaña - Wikipedia, la e...",La principal <b>causa de</b> esta aflicción .....,d4
...,...,...,...,...,...,...,...,...,...,...
114,6715485,335,yükseklik hastalığı,2017-07-22 10:46:22.0,2017-07-22 10:51:29.0,2.00,http://www.youtube.com/watch?v=vaC9Ch4eOrA,<b>Akut Dağ Hastalığı (Yükseklik Hastalığı) Ne...,<b>Akut Dağ Hastalığı (Yükseklik Hastalığı) Ne...,d35
115,6715485,335,yükseklik hastalığı,2017-07-22 10:46:22.0,2017-07-22 10:46:52.0,5.00,https://www.evrensel.net/yazi/75967/yukseklik-...,"<b>Yükseklik hastalığı</b>, savaşlar ve devrim...",İrtifa (<b>yükseklik) hastalığı</b> yüksek rak...,d36
116,40667959,423,"высотная болезнь симптомы, причины и предотвра...",2017-07-22 12:36:39.0,2017-07-22 12:37:02.0,2.00,http://www.erom.ru/zabol/zabol_v/vysotnaya_bol...,<b>Высотная</b> <b>болезнь</b>. (<b>Симптомы</...,<b>Симптомы и</b> лечение <b>высотной болезни<...,d37
117,40062468,315,el mal de altura,2017-07-22 10:30:47.0,2017-07-22 10:31:06.0,2.00,http://www.bbc.com/mundo/noticias/2016/05/1605...,¿Qué es <b>el mal de altura</b> que enluta la ...,La muerte <b>de</b> tres montañistas en <b>el<...,d38


107
size of beh_nasa 107
index is: 0
41683720
First time user
index is: 1
11311952
First time user
index is: 2
41683720
First time user
Count is 0
index is: 3
41683720
First time user
index is: 4
43665081
First time user
43665081


C:\Users\dimae\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\dimae\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\dimae\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

8    15093
Name: active_time, dtype: int64

...

,0,1,2,3,4
userID,,,,,
6335542,d,d2,NaN,NaN,NaN
6715485,d,d14,NaN,NaN,NaN
11582908,d,d9,d10,NaN,NaN
13396426,d,d2,NaN,NaN,NaN
15505671,d,d2,d6,d7,NaN
19298085,d,d2,NaN,NaN,NaN
25864227,d,d1,d2,NaN,NaN
31511492,d,d2,d5,d6,d14
33088361,d,d2,NaN,NaN,NaN


<ipython-input-101-d4176f290261>:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  behaviour_nasa["docgain"][i] = (urlindex['coef'][behaviour_nasa["urlid"][i]]).copy()
<ipython-input-101-d4176f290261>:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  behaviour_nasa["gain"][i] = (urlindex['coef'][behaviour_nasa["urlid"][i]] * behaviour_nasa["active_time"][i]).copy()


Here 2


,queryterm,urlid
0,tornado,"d2,d5,d6,d7,d13,d14"
1,tornado hurricane,d1
2,tornado rotation direction,d2
3,tornado word from,d8
4,tornado',d2
5,tornados espanol,d3
6,tornados espanol wikipedia,d4
7,weather phenomenon that is called tornado,"d9,d10,d11"
8,weather phenomenon tornado,d11
9,"погодном явлении, которое называется «торнадо»","d15,d16,d17,d18"


,Ndoc,R2,MSE,MAE
AltS,27,0.36,133.99,7.92
ARWar,23,0.60,217.37,10.88
Bees,25,0.33,492.91,17.11
Bombing,12,0.17,502.53,17.42
Evolution,45,0.32,120.20,7.49
NASA,30,0.45,189.20,9.72
Orcas,21,0.33,527.74,18.30
Sangre,18,0.33,334.60,14.86
SunTzu,34,0.44,294.41,11.89
Tornado,18,0.43,214.41,11.49


In [102]:
# import math 

display(regressionstat)
regressionstat["Overall"] =regressionstat.mean().round(2)
regressionstat["Ndoc"] = " & " + regressionstat["Ndoc"] + " & "
regressionstat["R2"] = " & " + str(regressionstat["R2"]) + " & "
regressionstat["MSE"] = " & " + str(regressionstat["MSE"]) + " & "
regressionstat["MAE"] = " & " + str(regressionstat["MAE"]) + " \\"

display(regressionstat)

,Ndoc,R2,MSE,MAE
AltS,27,0.36,133.99,7.92
ARWar,23,0.60,217.37,10.88
Bees,25,0.33,492.91,17.11
Bombing,12,0.17,502.53,17.42
Evolution,45,0.32,120.20,7.49
NASA,30,0.45,189.20,9.72
Orcas,21,0.33,527.74,18.30
Sangre,18,0.33,334.60,14.86
SunTzu,34,0.44,294.41,11.89
Tornado,18,0.43,214.41,11.49


,Ndoc,R2,MSE,MAE,Overall
AltS,& 27 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
ARWar,& 23 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
Bees,& 25 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
Bombing,& 12 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
Evolution,& 45 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
NASA,& 30 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
Orcas,& 21 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
Sangre,& 18 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
SunTzu,& 34 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN
Tornado,& 18 &,& AltS 0.36\nARWar ...,& AltS 133.99\nARWar ...,& AltS 7.92\nARWar ...,NaN


In [103]:
# import math 

# display(regressionstat)
regressionstat["Overall"] =regressionstat.mean().round(2)
regressionstat["Ndoc"] = " & " + regressionstat["Ndoc"] + " & "
regressionstat["R2"] = " & " + str(regressionstat["R2"]) + " & "
regressionstat["MSE"] = " & " + str(regressionstat["MSE"]) + " & "
regressionstat["MAE"] = " & " + str(regressionstat["MAE"]) + " \\"

display(regressionstat)

,Ndoc,R2,MSE,MAE,Overall
AltS,& & 27 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
ARWar,& & 23 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
Bees,& & 25 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
Bombing,& & 12 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
Evolution,& & 45 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
NASA,& & 30 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
Orcas,& & 21 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
Sangre,& & 18 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
SunTzu,& & 34 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN
Tornado,& & 18 & &,& AltS & AltS ...,& AltS & AltS 1...,& AltS & AltS ...,NaN


In [104]:
display(behaviour_nasa[behaviour_nasa['userID']==38301725])
# for i in behaviour_nasa.index:
#     print(i)

,userID,queryID,queryterm,query_time,click_time,rank,url,title,desc,urlid,active_time,gain,docgain


In [105]:
active_time

196    115625
Name: active_time, dtype: int64